### Build a Simple LLM Application with LCEL
In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

After seeing this video, you'll have a high level overview of:

- Using language models

- Using PromptTemplates and OutputParsers

- Using LangChain Expression Language (LCEL) to chain components together

- Debugging and tracing your application using LangSmith

- Deploying your application with LangServe

In [1]:
!pip install langchain

In [2]:
### Open AI API Key and Open Source models--Llama3,Gemma2,mistral--Groq

import os
from dotenv import load_dotenv
load_dotenv()

import openai
openai.api_key=os.getenv("OPENAI_API_KEY")

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key

'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'

In [3]:
!pip install langchain_groq

In [4]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
model=ChatGroq(model="openai/gpt-oss-20b",groq_api_key=groq_api_key)
model

/Users/sauravyadav/Downloads/GenerativeAI/1-Basics+Of+Langchain/.conda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x12c73a0f0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x12d2dfe30>, model_name='openai/gpt-oss-20b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [5]:
!pip install langchain_core

In [6]:
from langchain_core.messages import HumanMessage,SystemMessage
#  ChatPromptTemplate can also be used instead of manually creating messages
messages=[
    SystemMessage(content="Translate the following from English to French"),
    HumanMessage(content="Hello How are you?")
]

result=model.invoke(messages)

In [7]:
result

AIMessage(content='Bonjour, comment vas‑tu?', additional_kwargs={'reasoning_content': 'The user says: "Hello How are you?" The developer instruction: Translate the following from English to French. So we translate "Hello How are you?" to French. Should we include punctuation? Probably "Bonjour, comment ça va ?" or "Bonjour, comment vas-tu ?" The simplest: "Bonjour, comment vas-tu?" Or "Bonjour, comment allez-vous?" But the original is "Hello How are you?" which is informal. So "Bonjour, comment vas-tu?" We\'ll answer with that.'}, response_metadata={'token_usage': {'completion_tokens': 117, 'prompt_tokens': 86, 'total_tokens': 203, 'completion_time': 0.131525928, 'prompt_time': 0.004096195, 'queue_time': 0.055521785, 'total_time': 0.135622123, 'completion_tokens_details': {'reasoning_tokens': 101}}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_e99e93f2ac', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--0f0cf4d6-526e-4863-b629-c5b25d30

In [8]:
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()
parser.invoke(result)

'Bonjour, comment vas‑tu?'

In [9]:
### Using LCEL- chain the components
chain=model|parser
chain.invoke(messages)

'Bonjour, comment vas-tu?'

In [10]:
### Prompt Templates
from langchain_core.prompts import ChatPromptTemplate

generic_template="Trnaslate the following into {language}:"

prompt=ChatPromptTemplate.from_messages(
    [("system",generic_template),("user","{text}")]
)



In [11]:
result=prompt.invoke({"language":"French","text":"Hello"})
result

ChatPromptValue(messages=[SystemMessage(content='Trnaslate the following into French:', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})])

In [12]:
result.to_messages()

[SystemMessage(content='Trnaslate the following into French:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})]

In [13]:
##Chaining together components with LCEL
chain=prompt|model|parser
answer = chain.invoke({"language":"French","text":"Hello"})
answer


'Bonjour'

In [14]:
!pip install streamlit